In [2]:
#keras
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 40
epochs = 2

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),input_shape=input_shape))
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(128, activation='relu'))

model.add(Dropout(0.5))

model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 125s - loss: 0.4143 - acc: 0.8806 - val_loss: 0.0912 - val_acc: 0.9713
Epoch 2/2
60000/60000 [==============================] - 127s - loss: 0.1333 - acc: 0.9616 - val_loss: 0.0593 - val_acc: 0.9818
Test loss: 0.059266823971
Test accuracy: 0.9818


In [3]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    # Reshape X to 4-D tensor: [batch_size, width, height, channels]
    # MNIST images are 28x28 pixels, and have one color channel
    input_layer = tf.reshape(features["x"], [-1, 64, 64, 1])
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    conv1 = tf.nn.lrn(conv1)
    conv1 = tf.layers.batch_normalization(conv1)
    conv3 = tf.layers.conv2d(
      inputs=conv1,
      filters=32,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)
    
    conv5 = tf.layers.conv2d(
      inputs=conv3,
      filters=32,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)
    #conv2 = tf.layers.batch_normalization(conv2)
    
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
    pool1 = tf.nn.lrn(pool1)
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=32,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)
    
    conv4 = tf.layers.conv2d(
      inputs=conv2,
      filters=32,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)
    
    conv2 = tf.nn.lrn(conv2)
    conv2 = tf.layers.batch_normalization(conv2)
    pool2 = tf.layers.max_pooling2d(inputs=conv4, pool_size=[2, 2], strides=2)

    # pool2 = tf.nn.lrn(pool2)
    pool2_flat = tf.reshape(pool2, [-1, 16 * 16 * 32])
    pool2_flat = tf.reshape(pool1, [-1, 16 * 16 * 32])

    dense = tf.layers.dense(inputs=pool2_flat, units=512, activation=tf.nn.relu)
#     dense=tf.layers.batch_normalization(dense)
#     dense2=tf.layers.batch_normalization(dense2)
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.5, training=mode == tf.estimator.ModeKeys.TRAIN)
    logits = tf.layers.dense(inputs=dropout, units=40)

    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=40)
    loss = tf.losses.softmax_cross_entropy(
      onehot_labels=onehot_labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [6]:
import tensorflow as tf
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images  # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images  # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

NameError: name 'tf' is not defined

In [5]:
#Create estimator
mnist_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn,model_dir="/tmp/comp551_cnn_model21")

#Set up Logging Hook
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=100)

#Start training model
x_train = np.float32(x_train)
x_test = np.float32(x_test)
train_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x":x_train},y=y_train,batch_size=100,num_epochs=20 ,shuffle=False)
mnist_classifier.train(input_fn=train_input_fn,hooks=[logging_hook])

#Start evaluating model 
eval_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x":x_test},y=y_test,num_epochs=1,shuffle=False)
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)
                     
    
    
#Create estimator
mnist_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn,model_dir="/tmp/mnist_convnet_model_library2")

#Set up Logging Hook
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=100)

#Start training model
x_train = np.float32(x_train)
x_test = np.float32(x_test)
train_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x":train_data},y=train_labels,batch_size=100,num_epochs=None,shuffle=True)
mnist_classifier.train(input_fn=train_input_fn,steps=5000,hooks=[logging_hook])

#Start evaluating model 
eval_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x":eval_data},y=eval_labels,num_epochs=1,shuffle=False)
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)
                        

NameError: name 'tf' is not defined